Este archivo principal une todos los módulos y ejecuta el entrenamiento, validación y evaluación del modelo GCN.

In [16]:
#Importamos los modulos necesarios

import torch
from torch_geometric.loader import DataLoader
from Dataset_loader import PCBDataset  # Asegúrate que Dataset_loader se encuentra en el mismo directorio
from gcn_model import GCN
from data_loader import load_pcb_dataset
from train_eval import train, evaluate
import os

In [17]:
# Configuración del dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Mostrar mensaje de inicio
print("Iniciando el script...")
print(f"Usando dispositivo: {device}")

Iniciando el script...
Usando dispositivo: cpu


In [18]:
# Ruta al archivo de dataset
file_path = 'PCB_dataset.pth'  # Asegúrate que este archivo está en la carpeta actual

# Cargar el dataset
graphs = load_pcb_dataset(file_path)
print(f"Dataset cargado correctamente. Número de gráficos: {len(graphs)}")


c:\Users\Ernesto_Beca\Desktop\Ernesto\Repo_TFG\ia_thermal\ernestoemedina\data_loader.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load(file_path)


Dataset cargado correctamente. Número de gráficos: 20000


In [19]:
# División del dataset
train_size = int(0.8 * len(graphs))
val_size = int(0.1 * len(graphs))
test_size = len(graphs) - train_size - val_size

train_dataset = graphs[:train_size]
val_dataset = graphs[train_size:train_size + val_size]
test_dataset = graphs[train_size + val_size:]

# Crear DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [20]:
# Definición del modelo
input_dim = 2  # Potencia e interfaz como características
hidden_dim = 64
output_dim = 1  # Temperatura predicha por cada nodo
num_layers = 3
learning_rate = 0.001
epochs = 10

# Crear el modelo
model = GCN(input_dim, hidden_dim, output_dim, num_layers).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print("Modelo creado correctamente.")


Modelo creado correctamente.


In [21]:
# Entrenamiento del modelo
print("Iniciando entrenamiento...")
for epoch in range(epochs):
    train_loss = train(model, train_loader, optimizer, device)
    val_mse, val_mae, val_r2 = evaluate(model, val_loader, device)

    print(f"Epoch {epoch+1}/{epochs} - Loss: {train_loss:.4f} - Val MSE: {val_mse:.4f} - Val MAE: {val_mae:.4f} - Val R2: {val_r2:.4f}")


Iniciando entrenamiento...
Epoch 1/10 - Loss: 68231.2293 - Val MSE: 678.4055 - Val MAE: 20.8668 - Val R2: 0.4769
Epoch 2/10 - Loss: 559.3938 - Val MSE: 542.3721 - Val MAE: 18.6498 - Val R2: 0.5818
Epoch 3/10 - Loss: 520.0184 - Val MSE: 522.6764 - Val MAE: 18.3184 - Val R2: 0.5970
Epoch 4/10 - Loss: 510.1850 - Val MSE: 518.7855 - Val MAE: 18.1830 - Val R2: 0.6000
Epoch 5/10 - Loss: 509.3140 - Val MSE: 520.0098 - Val MAE: 18.2102 - Val R2: 0.5990
Epoch 6/10 - Loss: 509.1576 - Val MSE: 517.9576 - Val MAE: 18.1642 - Val R2: 0.6006
Epoch 7/10 - Loss: 509.3623 - Val MSE: 520.0292 - Val MAE: 18.2306 - Val R2: 0.5990
Epoch 8/10 - Loss: 508.7480 - Val MSE: 518.2868 - Val MAE: 18.1627 - Val R2: 0.6004
Epoch 9/10 - Loss: 509.6545 - Val MSE: 518.0912 - Val MAE: 18.1764 - Val R2: 0.6005
Epoch 10/10 - Loss: 509.2879 - Val MSE: 518.6553 - Val MAE: 18.1879 - Val R2: 0.6001


In [25]:
# Evaluación en el conjunto de prueba
print("Evaluando en el conjunto de prueba...")
error_threshold = 5.0  # Cambia esto al valor que desees en Kelvin

test_mse, test_mae, test_r2, test_accuracy = evaluate(model, test_loader, device, error_threshold=5.0)


print(f"Test MSE: {test_mse:.4f} - Test MAE: {test_mae:.4f} - Test R2: {test_r2:.4f}")
print(f"Porcentaje de nodos correctamente predichos dentro de un error de ±{error_threshold} K: {test_accuracy:.2f}%")



Evaluando en el conjunto de prueba...


TypeError: evaluate() got an unexpected keyword argument 'error_threshold'